In [1]:
import numpy as np
import soundfile as sf
from scipy.interpolate import interp1d
import pandas as pd
from scipy.fft import rfft, irfft, rfftfreq
from scipy.signal import welch
import matplotlib.pyplot as plt
import os
import gc
import glob

# Paths
base_path = "C:/Users/Valer/OneDrive/ODL Hydrophone/ODL Recordings/"
output_csv_folder = "C:/Users/Valer/OneDrive/ODL Hydrophone/Data Sheets/PSD Sheets/"
output_fig_psd_folder = "C:/Users/Valer/OneDrive/ODL Hydrophone/Data Sheets/PSD Figures/"
calib_csv_path = "C:/Users/Valer/OneDrive/ODL Hydrophone/Data Sheets/Bins Data for Hydrophone Sensitivity.csv"

# Make sure output folders exist
os.makedirs(output_csv_folder, exist_ok=True)
os.makedirs(output_fig_psd_folder, exist_ok=True)

# Load calibration data
calib_df = pd.read_csv(calib_csv_path)
freq_calib = calib_df["Frequency (Hz)"].to_numpy()
sens_calib = calib_df["Sensitivity (dB re 1V/1uPa)"].to_numpy()
sens_calib_gain = sens_calib + 25.0

# Process WAV files
for filename in os.listdir(base_path):
    if filename.lower().endswith(".wav"):
        output_filename = f"psd_output_{os.path.splitext(filename)[0]}.csv"
        output_path = os.path.join(output_csv_folder, output_filename)
        if os.path.exists(output_path):
            print(f" Skipping already processed file: {filename}")
            continue

        print(f"Processing: {filename}")
        wav_path = os.path.join(base_path, filename)
        signal_raw, fs = sf.read(wav_path)
        N = len(signal_raw)
        freqs = rfftfreq(N, d=1/fs)

        # Interpolate and apply calibration
        interp_func = interp1d(freq_calib, sens_calib_gain, kind='linear', fill_value='extrapolate')
        sens_interp_dB = interp_func(freqs)
        gain_correction = 10 ** (-sens_interp_dB / 20)
        signal_fft = rfft(signal_raw)
        calibrated_fft = signal_fft * gain_correction
        signal_calibrated = irfft(calibrated_fft)

        # Welch PSD
        L = 65536
        n_overlap = int(L * 0.5)
        freqs_psd, psd = welch(signal_calibrated, fs=fs, window='hann', nperseg=L,
                               noverlap=n_overlap, nfft=L, scaling='density')
        psd_dB = 10 * np.log10(psd)

        # Save PSD CSV
        df_psd_db = pd.DataFrame({
            "Frequency (Hz)": freqs_psd,
            "PSD (dB re 1 µPa²/Hz)": psd_dB
        })
        df_psd_db.to_csv(output_path, index=False)
        print(f" PSD saved to: {output_path}")

        # Plot and save PSD
        fig_psd, ax_psd = plt.subplots(figsize=(12, 6))
        ax_psd.semilogx(freqs_psd, psd_dB)
        ax_psd.set_title(f"PSD of {filename}")
        ax_psd.set_xlabel("Frequency (Hz)")
        ax_psd.set_ylabel("PSD (dB re 1 µPa²/Hz)")
        ax_psd.grid(True, which='both', linestyle='--')
        ax_psd.set_xlim(1, fs / 2)
        plt.tight_layout()

        psd_plot_name = f"psd_{os.path.splitext(filename)[0]}.png"
        psd_plot_path = os.path.join(output_fig_psd_folder, psd_plot_name)
        plt.savefig(psd_plot_path)
        plt.close()
        gc.collect()  # free memory of RAM (use it to handle the situation that it kept crashing)
        print(f"PSD plot saved to: {psd_plot_path}\n")

        # Free memory (troubleshooting ram issue)
        del signal_raw, signal_fft, calibrated_fft, signal_calibrated
        del psd_dB, psd, df_psd_db
        del fig_psd, ax_psd

 Skipping already processed file: 221019_0052.wav
 Skipping already processed file: 221019_0053.wav
 Skipping already processed file: 221019_0054.wav
 Skipping already processed file: 221019_0055.wav
 Skipping already processed file: 221019_0056.wav
 Skipping already processed file: 221019_0057.wav
 Skipping already processed file: 221019_0058.wav
 Skipping already processed file: 221019_0059.wav
 Skipping already processed file: 221019_0060.wav
 Skipping already processed file: 221019_0061.wav
 Skipping already processed file: 221019_0062.wav
 Skipping already processed file: 221019_0063.wav
 Skipping already processed file: 221019_0064.wav
 Skipping already processed file: 221019_0065.wav
 Skipping already processed file: 221019_0066.wav
 Skipping already processed file: 221019_0067.wav
 Skipping already processed file: 221019_0068.wav
 Skipping already processed file: 221019_0069.wav
 Skipping already processed file: 221019_0070.wav
 Skipping already processed file: 221019_0071.wav


In [ ]:
# Combine all separate PSD into one CSV (THE MASTER PSD HIHI)
all_psd_csvs = glob.glob(os.path.join(output_csv_folder, "psd_output_*.csv"))
df_all_psds = pd.DataFrame()

for csv_file in all_psd_csvs:
    df_psd = pd.read_csv(csv_file)
    recording_id = os.path.splitext(os.path.basename(csv_file))[0].replace("psd_output_", "")
    df_psd_renamed = df_psd.rename(columns={"PSD (dB re 1 µPa²/Hz)": f"PSD_{recording_id} (dB)"})
    
    if df_all_psds.empty:
        df_all_psds = df_psd_renamed
    else:
        df_all_psds = pd.merge(df_all_psds, df_psd_renamed, on="Frequency (Hz)", how="outer")

# Save combined master CSV
combined_csv_path = os.path.join(output_csv_folder, "ALL_PSDs_combined.csv")
df_all_psds.to_csv(combined_csv_path, index=False)
print(f"Combined all PSDs into: {combined_csv_path}")
